In [1]:
import os, sys
sys.path.append('../')
import pandas as pd
import geopandas as gpd
from utils.modeling import fit_density_model

In [2]:
pth = os.path.join('..', 'data', 'geodataframe.gpkg')
gdf = gpd.read_file(pth); gdf

,city,state,country,latitude,longitude,population,msa,US,geometry
0,mahopac,new york,united states,41.37232,-73.73346,8369,New York-Northern New Jersey-Long Island NY-NJ...,True,POINT (-8207971.222 5067414.863)
1,seven hills,ohio,united states,41.39533,-81.67624,11690,None,True,POINT (-9092157.447 5070828.79)
2,new martinsville,west virginia,united states,39.64452,-80.85760,5218,None,True,POINT (-9001026.859 4814418.565)
3,sint anthonis,north brabant,netherlands,51.62667,5.88194,3340,None,False,POINT (654774.566 6732902.058)
4,machali,o higgins region,chile,-34.18082,-70.64933,27595,None,False,POINT (-7864647.44 -4053107.653)
...,...,...,...,...,...,...,...,...,...
24022,mansfield,texas,united states,32.56319,-97.14168,64274,Mansfield OH MSA,True,POINT (-10813762.352 3837467.237)
24023,belfast,maine,united states,44.42591,-69.00642,6682,None,True,POINT (-7681759.536 5531591.056)
24024,flushing,michigan,united states,43.06308,-83.85107,8086,Wheeling WV-OH MSA,True,POINT (-9334258.415 5321578.2)
24025,stone mountain,georgia,united states,33.80816,-84.17020,6109,Atlanta-Sandy Springs-Marietta GA MSA,True,POINT (-9369783.804 4003071.597)


In [3]:
# world equidistant projection (x,y)
crs = gdf.crs
gdf.to_crs('EPSG:4087', inplace = True)
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
gdf['w'] = gdf.population
gdf['cluster'] = -1
gdf.to_crs(crs, inplace = True); gdf

,city,state,country,latitude,longitude,population,msa,US,geometry,x,y,w,cluster
0,mahopac,new york,united states,41.37232,-73.73346,8369,New York-Northern New Jersey-Long Island NY-NJ...,True,POINT (-8207971.222 5067414.863),-8.207971e+06,4.605546e+06,8369,-1
1,seven hills,ohio,united states,41.39533,-81.67624,11690,None,True,POINT (-9092157.447 5070828.79),-9.092157e+06,4.608107e+06,11690,-1
2,new martinsville,west virginia,united states,39.64452,-80.85760,5218,None,True,POINT (-9001026.859 4814418.565),-9.001027e+06,4.413208e+06,5218,-1
3,sint anthonis,north brabant,netherlands,51.62667,5.88194,3340,None,False,POINT (654774.566 6732902.058),6.547746e+05,5.747055e+06,3340,-1
4,machali,o higgins region,chile,-34.18082,-70.64933,27595,None,False,POINT (-7864647.44 -4053107.653),-7.864647e+06,-3.804991e+06,27595,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24022,mansfield,texas,united states,32.56319,-97.14168,64274,Mansfield OH MSA,True,POINT (-10813762.352 3837467.237),-1.081376e+07,3.624918e+06,64274,-1
24023,belfast,maine,united states,44.42591,-69.00642,6682,None,True,POINT (-7681759.536 5531591.056),-7.681760e+06,4.945470e+06,6682,-1
24024,flushing,michigan,united states,43.06308,-83.85107,8086,Wheeling WV-OH MSA,True,POINT (-9334258.415 5321578.2),-9.334258e+06,4.793760e+06,8086,-1
24025,stone mountain,georgia,united states,33.80816,-84.17020,6109,Atlanta-Sandy Springs-Marietta GA MSA,True,POINT (-9369783.804 4003071.597),-9.369784e+06,3.763507e+06,6109,-1


In [ ]:
# density-based clustering
eps_dbscan = 30e3              # 30 km max distance
min_samples_dbscan = int(1e6)  # 1m min population
min_samples_decay = .98
keep_clustering = True
while keep_clustering:

    # fit model (unclustered only)
    clus_msk = gdf.cluster >= 0
    nclus0 = gdf.loc[clus_msk].cluster.nunique()
    gdf_sub = gdf.loc[~clus_msk].copy()
    gdf_sub = fit_density_model(gdf_sub, eps_dbscan, min_samples_dbscan)
    clus_msk_sub = gdf_sub.cluster >= 0
    gdf_sub.loc[clus_msk_sub, 'cluster'] += max(gdf.cluster.max(), 0)
    gdf.update(gdf_sub)

    # check convergence
    clus_msk = gdf.cluster >= 0
    nclus1 = gdf.loc[clus_msk].cluster.nunique()
    print('Number of clusters =', nclus1)
    if nclus1 > nclus0:
        min_samples_dbscan = int(min_samples_dbscan * min_samples_decay)
    else:
        keep_clustering = False

Number of clusters = 268
Number of clusters = 273
Number of clusters = 277
Number of clusters = 280
Number of clusters = 287
Number of clusters = 289
Number of clusters = 295
Number of clusters = 301
Number of clusters = 303
Number of clusters = 307
Number of clusters = 316
Number of clusters = 321
Number of clusters = 325
Number of clusters = 336
Number of clusters = 341
Number of clusters = 346
Number of clusters = 354
Number of clusters = 355
Number of clusters = 360
Number of clusters = 364
Number of clusters = 368
Number of clusters = 376
Number of clusters = 379
Number of clusters = 384
Number of clusters = 389
Number of clusters = 393
Number of clusters = 399
Number of clusters = 404
Number of clusters = 407
Number of clusters = 410
Number of clusters = 413
Number of clusters = 418
Number of clusters = 426
Number of clusters = 435
Number of clusters = 441
Number of clusters = 444
Number of clusters = 451
Number of clusters = 456
Number of clusters = 461
Number of clusters = 466


In [5]:
# assign cluster names (biggest city)
cluster_names = (gdf[gdf.cluster >= 0]
                 .sort_values('population', ascending = False)
                 .drop_duplicates('cluster')
                 .set_index('cluster')
                 .apply(lambda row: f'{row.city} {row.state} {row.country}', axis = 1))
gdf['cluster_name'] = gdf.cluster.map(cluster_names)
gdf.loc[gdf.cluster < 0, 'cluster_name'] = pd.NA
gdf['cluster'] = gdf.cluster_name.astype('string')
gdf.drop(columns = ['cluster_name', 'x', 'y', 'w'], inplace = True); gdf

,city,state,country,latitude,longitude,population,msa,US,geometry,cluster
0,mahopac,new york,united states,41.37232,-73.73346,8369,New York-Northern New Jersey-Long Island NY-NJ...,True,POINT (-8207971.222 5067414.863),new york city new york united states
1,seven hills,ohio,united states,41.39533,-81.67624,11690,None,True,POINT (-9092157.447 5070828.79),cleveland ohio united states
2,new martinsville,west virginia,united states,39.64452,-80.85760,5218,None,True,POINT (-9001026.859 4814418.565),<NA>
3,sint anthonis,north brabant,netherlands,51.62667,5.88194,3340,None,False,POINT (654774.566 6732902.058),venray limburg netherlands
4,machali,o higgins region,chile,-34.18082,-70.64933,27595,None,False,POINT (-7864647.44 -4053107.653),<NA>
...,...,...,...,...,...,...,...,...,...,...
24022,mansfield,texas,united states,32.56319,-97.14168,64274,Mansfield OH MSA,True,POINT (-10813762.352 3837467.237),dallas texas united states
24023,belfast,maine,united states,44.42591,-69.00642,6682,None,True,POINT (-7681759.536 5531591.056),<NA>
24024,flushing,michigan,united states,43.06308,-83.85107,8086,Wheeling WV-OH MSA,True,POINT (-9334258.415 5321578.2),flint michigan united states
24025,stone mountain,georgia,united states,33.80816,-84.17020,6109,Atlanta-Sandy Springs-Marietta GA MSA,True,POINT (-9369783.804 4003071.597),atlanta georgia united states


In [7]:
gdf.to_file(os.path.join('..', 'data', 'geodataframe_dbscan.gpkg'), driver = 'GPKG')